<a href="https://colab.research.google.com/github/PaulQuezada/InteligenciaArtificial/blob/main/8-puzzle/8_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**8 Puzzle**
1. ¿Que es? \
El 8-puzzle es un popular rompecabezas deslizante que se juega en una cuadrícula 3x3. El rompecabezas consiste en una cuadrícula con 8 fichas numeradas y un espacio vacío, lo que da un total de 9 posiciones. El objetivo del juego es reorganizar las fichas numeradas en orden ascendente, generalmente desde 1 hasta 8, dejando el espacio vacío en la última posición.

#**Desarrollando el puzzle con busqueda en anchura**

Primero, para desarrollar el puzzle con BFS, lo primero que tenemos que definir sera la matriz, la cual la estableceremos de la siguiente manera. \
Este sera nuestro tablero objetivo, el cual sera aleatorio, es posible sino revisamos el tablero, que el tablero objetivo no sea solucionable, entonces verificaremos el tablero hasta que sea valido con la funcion que se mostrara a continuacion.

In [7]:
import random

def es_soluble(tablero):
    aplanado = [num for fila in tablero for num in fila if num != "_"]
    inversiones = 0

    for i in range(len(aplanado)):
        for j in range(i + 1, len(aplanado)):
            if aplanado[i] > aplanado[j]:
                inversiones += 1

    return inversiones % 2 == 0

def desordenar_tablero(tablero):
    aplanado = [num for fila in tablero for num in fila]
    random.shuffle(aplanado)
    nuevo_tablero = [aplanado[i:i+3] for i in range(0, 9, 3)]

    intentos = 0
    while not es_soluble(nuevo_tablero):
        intentos += 1
        print(f"Intento {intentos}: El tablero no es soluble, creando uno nuevo...")
        random.shuffle(aplanado)
        nuevo_tablero = [aplanado[i:i+3] for i in range(0, 9, 3)]

    return nuevo_tablero
tablero_inicial = [[1, 2, 3],
                  [5, 6, "_"],
                  [7, 8, 4]]

tablero_objetivo = desordenar_tablero(tablero_inicial)
print("Tablero objetivo: ",tablero_objetivo)

Tablero objetivo:  [[3, 2, 4], [6, 5, 8], ['_', 7, 1]]


Luego, estableceremos un tablero inicial, al principio lo habia hecho aleatorio, pero hay un problema con esto, debido a que no todos los tableros iniciales del 8-puzzle son solucionables, por lo cual, declararemos un tablero inicial que contenga al menos 1 solucion.


In [8]:
tablero_inicial = [[1, 2, 3],
                  [5, 6, "_"],
                  [7, 8, 4]]

Las librerias que se listaran a continuacion son importantes por las siguientes razones: \
1. Libreria time: Nos servira para poder medir el tiempo que se desmora el algoritmo en encontrar la solucion.
2. Libreria deque: deque es un tipo de listado en donde ponemos insertar valores al comienzo o final de la lista. Es como una fusion entre las colas y las pilas. Es importante en este caso para utilizar la cola en el algoritmo.

In [9]:
import time
from collections import deque

## **Clase Nodo**

Crearemos una clase llamado **Nodo**, cada nodo contendra:
1. **El tablero**
2. **El padre**
3. **La accion** que se desarrollo para llegar al tablero que contiene \
**Observacion:** El nodo inicial no contiene padre.

## **Funciones de la clase Nodo**
1. **init**: Se instancian los datos del nodo con la clase
2. **obtener_vacio**: su unica funcion es mostrar la posicion del espacio en el tablero.
3. **sucesores**: Esta, buscara todos los sucesores posibles del tablero del nodo y los retornara en una lista.

In [10]:
class Nodo:
    # Clase que representa un nodo en el arbol de busqueda
    def __init__(self, tablero, padre=None, accion=None):
        self.tablero = tablero
        self.padre = padre
        self.accion = accion
    # Metodo para obtener la posicion del espacio en el tablero
    def obtener_vacio(self):
        for i in range(3):
            for j in range(3):
                if self.tablero[i][j] == '_':
                    return (i, j)

    # Metodo para obtener los sucesores validos del tablero
    def sucesores(self):
        i, j = self.obtener_vacio()
        movimientos = [
            (0, 1), # Moviemiento hacia Derecha
            (1, 0), # Moviemiento hacia Abajo
            (0, -1),# Moviemiento hacia Izquierda
            (-1, 0)]# Moviemiento hacia Arriba

        # Generamos los sucesores
        sucesores = []

        for di, dj in movimientos:
            # Validamos que el movimiento este dentro del tablero
            ni = i+di
            nj = j+dj
            if 0 <= ni < 3 and 0 <= nj < 3:
                # Si el movimiento es valido, generamos un nuevo tablero y lo agregamos a la lista de los sucesores
                nuevo_tablero = [fila.copy() for fila in self.tablero]
                nuevo_tablero[i][j], nuevo_tablero[ni][nj] = nuevo_tablero[ni][nj], nuevo_tablero[i][j]
                sucesores.append(Nodo(nuevo_tablero, self, (ni, nj)))

        return sucesores

## **Funcion BFS**

En esta funcion, crearemos un conjunto para almacenar los tableros visitiados (Lo realizo con **set()**, debido a que es mucho mas facil buscar los valores repetidos). Tambien crearemos la cola, en donde guardaremos los nodos que generen los movimientos posibles a lo cual llamamos **sucesores**. \

Recorreremos los sucesores del nodo actual, los agregaremos a nuestro conjunto de visitados, verificamos si este nodo no ha sido recorrido, en caso de que este no haya sido recorrido lo agregaremos a la cola para posteriormente calculas sus sucesores hasta encontrar una solucion. \


Por naturaleza, el algoritmo BFS siempre encontrara primero la solucion mas corta. por lo que no es necesario recorrer todos las soluciones posibles y compararlas.


In [11]:
def BFS(inicio, objetivo):
    visitados = set() # Creamos un conjunto para almacenar los tableros visitados
    cola = deque([Nodo(inicio)]) # Creamos una cola para almacenar los nodos que se van a expandir

    while cola:
        nodo = cola.popleft() # Sacamos el primer nodo de la cola
        if nodo.tablero == objetivo: # Verificamos si el tablero del nodo es el objetivo
            return nodo # Si lo es, retornamos el nodo

        for sucesor in nodo.sucesores(): # Generamos los sucesores del nodo
            tablero_str = str(sucesor.tablero) # Convertimos el tablero a string para poderlo almacenar en el conjunto (Esto lo realizamos para hacer mas facil la comparacion entre los tableros)
            if tablero_str not in visitados: # Verificamos que el tablero no haya sido visitado
                visitados.add(tablero_str) # Si no ha sido visitado, lo agregamos al conjunto de visitados
                cola.append(sucesor) # Agregamos el sucesor a la cola

    return None

## **Ejecucion del algoritmo**

Por ultimo, ejecutaremos el algorimo. Al haber agregado el padre en cada nodo, con el ultimo nodo del camino podemos saber todo el recorrido y las acciones que se realizaron, por lo cual, recorreremos recursivamente hasta encontrar el nodo inicial, estos nodos los guardaremos en un arreglo llamado **camino** el cual le aplicaremos un **reserse** debido a que estamos recorriendo desde el nodo final hasta el inicial. \

Mostraremos ademas del camino que recorrio, los movimientos que se realizaron en total y el tiempo que se desmoro.

In [12]:
# Empezamos el cronometro
start_time = time.time()

# Ejecutamos el algoritmo BFS
resultado = BFS(tablero_inicial, tablero_objetivo)
# Si encontramos el camino lo imprimimos
if resultado:
    # Terminamos el cronometro
    elapsed_time = time.time() - start_time
    print("¡Solución encontrada!")
    camino = []
    while resultado: # Recorremos el camino desde el nodo objetivo hasta el nodo inicial
        camino.append(resultado.tablero)
        resultado = resultado.padre
    camino.reverse()
    for paso in camino:
        print("Movimiento numero:", camino.index(paso))
        for fila in paso:
            print(fila)
        print("--------------")
    print("Tiempo de ejecución: %.10f segundos." % elapsed_time)
    print("Número de movimientos: ", len(camino)-1)
else:
     # Terminamos el cronometro
    elapsed_time = time.time() - start_time
    print("Tiempo de ejecución: %.10f segundos." % elapsed_time)
    print("No se encontró solución.")

¡Solución encontrada!
Movimiento numero: 0
[1, 2, 3]
[5, 6, '_']
[7, 8, 4]
--------------
Movimiento numero: 1
[1, 2, '_']
[5, 6, 3]
[7, 8, 4]
--------------
Movimiento numero: 2
[1, '_', 2]
[5, 6, 3]
[7, 8, 4]
--------------
Movimiento numero: 3
[1, 6, 2]
[5, '_', 3]
[7, 8, 4]
--------------
Movimiento numero: 4
[1, 6, 2]
[5, 3, '_']
[7, 8, 4]
--------------
Movimiento numero: 5
[1, 6, 2]
[5, 3, 4]
[7, 8, '_']
--------------
Movimiento numero: 6
[1, 6, 2]
[5, 3, 4]
[7, '_', 8]
--------------
Movimiento numero: 7
[1, 6, 2]
[5, 3, 4]
['_', 7, 8]
--------------
Movimiento numero: 8
[1, 6, 2]
['_', 3, 4]
[5, 7, 8]
--------------
Movimiento numero: 9
['_', 6, 2]
[1, 3, 4]
[5, 7, 8]
--------------
Movimiento numero: 10
[6, '_', 2]
[1, 3, 4]
[5, 7, 8]
--------------
Movimiento numero: 11
[6, 3, 2]
[1, '_', 4]
[5, 7, 8]
--------------
Movimiento numero: 12
[6, 3, 2]
['_', 1, 4]
[5, 7, 8]
--------------
Movimiento numero: 13
[6, 3, 2]
[5, 1, 4]
['_', 7, 8]
--------------
Movimiento numero: 14


# **Desarrollando el puzzle con busqueda en profundidad**
Primero, para desarrollar el puzzle con DFS, en si, lo que cambia es la forma de buscar la solucion, para este problema en especifico NO es recomendable usar este tipo de busqueda, esto debido a que puede caer en un bucle infinito, un uso excesivo de memoria, entre otros.

El codigo sera practicamente el mismo, solo que cambiaremos la forma de como agregamos y quitamos elementos a nuestro conjunto, en este caso no usaremos una cola, sino, una cola.

A continuacion importamos las librerias.

In [14]:
import random
import time
from collections import deque

## **Declaramos el nodo inicial y objetivo**

Declararemos el mismo nodo(que contiene el tablero) inicial para comparar los tiempos. Tambien en este punto declararemos el conjunto de visitados, para que la funcion **obtener_sucesores** de la clase **Nodo** pueda comparar los nodos sucesores con los nodos explorados.

El tablero objetivo sera aleatorio igual que en el algoritmo de **BFS**.

In [15]:
import random

def es_soluble(tablero):
    aplanado = [num for fila in tablero for num in fila if num != "_"]
    inversiones = 0

    for i in range(len(aplanado)):
        for j in range(i + 1, len(aplanado)):
            if aplanado[i] > aplanado[j]:
                inversiones += 1

    return inversiones % 2 == 0

def desordenar_tablero(tablero):
    aplanado = [num for fila in tablero for num in fila]
    random.shuffle(aplanado)
    nuevo_tablero = [aplanado[i:i+3] for i in range(0, 9, 3)]

    intentos = 0
    while not es_soluble(nuevo_tablero):
        intentos += 1
        print(f"Intento {intentos}: El tablero no es soluble, creando uno nuevo...")
        random.shuffle(aplanado)
        nuevo_tablero = [aplanado[i:i+3] for i in range(0, 9, 3)]

    return nuevo_tablero
tablero_inicial = [[1, 2, 3],
                  [5, 6, "_"],
                  [7, 8, 4]]

tablero_objetivo = desordenar_tablero(tablero_inicial)
print("Tablero objetivo: ",tablero_objetivo)

visitados = set()

Tablero objetivo:  [['_', 7, 2], [8, 1, 4], [6, 5, 3]]


## **Clase Nodo**

Agregamos nuestra clase Nodo, es practicamente igual a la que se uso en BFS, solo que en este comparamos los sucesores con la lista de los visitados.

In [16]:
class Nodo:
    # Clase que representa un nodo en el arbol de busqueda
    def __init__(self, tablero=None, padre=None, accion=None):
        self.tablero = tablero
        self.padre = padre
        self.accion = accion
    # Metodo para obtener la posicion del espacio en el tablero
    def obtener_vacio(self):
        for i in range(3):
            for j in range(3):
                if self.tablero[i][j] == '_':
                    return (i, j)

    def obtener_sucesores(self):
        i, j = self.obtener_vacio()
        movimientos = [
            (0, 1), # Moviemiento hacia Derecha
            (1, 0), # Moviemiento hacia Abajo
            (0, -1),# Moviemiento hacia Izquierda
            (-1, 0) # Moviemiento hacia Arriba
        ]

        sucesores = []

        for di, dj in movimientos:
            ni = i + di
            nj = j + dj
            if 0 <= ni < 3 and 0 <= nj < 3:
                nuevo_tablero = [fila.copy() for fila in self.tablero]
                nuevo_tablero[i][j], nuevo_tablero[ni][nj] = nuevo_tablero[ni][nj], nuevo_tablero[i][j]
                tablero_str = str(nuevo_tablero)
                if tablero_str not in visitados:
                    visitados.add(tablero_str)
                    sucesores.append(Nodo(nuevo_tablero, self, (ni, nj)))

        return sucesores

## **Funcion DFS**

En este caso tendremos una pila, en donde en cada iteracion quitamos el primer elemento de la pila, luego, agregamos los sucesores al final de la fila, de forma, cuando algun nodo no contenga mas nodos no visitados, lo quitara de la pila y seguira con los sucesores que se generaron anteriormente, de esta forma, siempre recorrera hasta el final de un nodo, en donde continuara con los nodos anteriores que no fueron explorados todavia.

Con esta explicacion, podemos decir, que si existe una solucion la encontrara, el problema es a que costo, en el peor de los casos, recorrera todos los caminos posibles hasta encontrar la solucion, lo cual es costoso en cuanto a tiempo y memoria.

In [17]:
def DFS(inicio, objetivo):
    pila = deque([Nodo(inicio)])
    while pila:
        nodo = pila.pop()  # Pop obtiene y remueve el último elemento, que es el comportamiento de una pila.
        if nodo.tablero == objetivo:
            return nodo

        sucesores = nodo.obtener_sucesores()  # Cambiamos el método para que devuelva todos los sucesores.
        for sucesor in sucesores:
            pila.append(sucesor)

    return None

## **Ejecucion del algoritmo DFS**

Por ultimo, ejecutaremos el algoritmo. En este caso no mostrare los movimientos, porque pueden ser mucho mas grandes que el caminos mas corto, en este apartado podremos ver la cantidad de movimientos que se realizaron para resolver el puzzle y el tiempo que se desmoro.

In [18]:
# Empezamos el cronometro
start_time = time.time()
# Ejecutamos el algoritmo BFS
resultado = DFS(tablero_inicial, tablero_objetivo)
# Si encontramos el camino lo imprimimos
if resultado:
    # Terminamos el cronometro
    elapsed_time = time.time() - start_time
    print("¡Solución encontrada!")
    camino = []
    while resultado: # Recorremos el camino desde el nodo objetivo hasta el nodo inicial
        camino.append(resultado.tablero)
        resultado = resultado.padre
    print("Tiempo de ejecución: %.10f segundos." % elapsed_time)
    print("Número de movimientos: ", len(camino)-1)
else:
     # Terminamos el cronometro
    elapsed_time = time.time() - start_time
    print("Tiempo de ejecución: %.10f segundos." % elapsed_time)
    print("No se encontró solución.")


¡Solución encontrada!
Tiempo de ejecución: 1.7220132351 segundos.
Número de movimientos:  66019


# **Conlusiones**

Los resultados subrayan la importancia de seleccionar el algoritmo adecuado segun el contexto y los requisitos especificos del problema. Mientras que **BFS** proporcionó una solución más rápida y eficiente para el **8-puzzle** en este caso, **DFS** podría ser preferible en situaciones donde el espacio de memoria es una preocupación y la eficiencia de la solución no es crítica.